In [ ]:
import polars as pl
import networkx as nx
from math import sqrt
import random
import rtsvg
rt = rtsvg.RACETrack()

# Create a graph
from linknode_graph_patterns import LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()
g          = _patterns_.createPattern('mesh')

# Create the dataframe version of the graph
_lu_ = {'fm':[], 'to':[]}
for edge in g.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
df = pl.DataFrame(_lu_)

#
# "Graph Drawing by Force-directed Placement"
# Fruchterman & Reingold
# Sofftware -- Practice and Experience, Vol. 21 (1 1), 1129-1164 (November 1991)
#
# Figure 1 (Force-directed Placement)
#

# Parameters for the layout operation
t    = 3.0 # temperature
C    = 0.2 # paper says that C is found experimentally...
W, H = 256, 256
area = W*H
k    = C*sqrt(area/len(g.nodes()))

# Create the distance dataframe from the graph
_lu_    = {'fm':[], 'to':[], 't':[]}
for node in g.nodes:
    for nbor in g.neighbors(node):
        _w_ = g[node][nbor]['weight'] if 'weight' in g[node][nbor] else 1.0
        _lu_['fm'].append(node), _lu_['to'].append(nbor), _lu_['t'].append(_w_)
df_dist = pl.DataFrame(_lu_)

# Create positional dataframe w/ random values
_lu_    = {'node':[], 'x':[], 'y':[], 's':[]}
for node in g.nodes: _lu_['node'].append(node), _lu_['x'].append(W*random.random()), _lu_['y'].append(H*random.random()), _lu_['s'].append(False)
df_pos  = pl.DataFrame(_lu_)

positions = []
for i in range(2*len(g.nodes)):
    positions.append(df_pos)
    # Cross join so that all combinations of nodes are considered (except where nod and node_right are the same node)
    df_cross = df_pos.drop('s')\
                     .join(df_pos.drop('s'), how='cross') \
                     .filter(pl.col('node') != pl.col('node_right'))

    # Repulsive Forces
    __dx__, __dy__ = (pl.col('x') - pl.col('x_right')), (pl.col('y') - pl.col('y_right'))
    df_repulse = df_cross.join(df_dist, left_on=['node', 'node_right'], right_on=['fm', 'to'], how='anti') \
                         .with_columns((__dx__**2 + __dy__**2).sqrt().alias('d')) \
                         .with_columns(pl.when(pl.col('d') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('d')).alias('d')) \
                         .with_columns(((k**2)/pl.col('d')).alias('f_r')) \
                         .with_columns(((__dx__/pl.col('d')) * pl.col('f_r')).alias('disp_x'),
                                       ((__dy__/pl.col('d')) * pl.col('f_r')).alias('disp_y'))

    # Attractive Forces
    df_attract = df_cross.join(df_dist, left_on=['node', 'node_right'], right_on=['fm', 'to']) \
                         .with_columns((__dx__**2 + __dy__**2).sqrt().alias('d')) \
                         .with_columns(pl.when(pl.col('d') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('d')).alias('d')) \
                         .with_columns(((pl.col('d')**2)/k).alias('f_a')) \
                         .with_columns((-(__dx__)/(pl.col('d')*pl.col('t'))*pl.col('f_a')).alias('disp_x'),
                                       (-(__dy__)/(pl.col('d')*pl.col('t'))*pl.col('f_a')).alias('disp_y'),
                                       (  __dx__ /(pl.col('d')*pl.col('t'))*pl.col('f_a')).alias('disp_x_right'),
                                       (  __dy__ /(pl.col('d')*pl.col('t'))*pl.col('f_a')).alias('disp_y_right')) \
                         .drop({'x','y','x_right','y_right'})

    # Sum them up
    df_sums = pl.concat([df_repulse.drop({'x','y','node_right','x_right','y_right','d','f_r'}),
                         df_attract.drop({'node_right','t','d','f_a','disp_x_right','disp_y_right'}),
                         df_attract.drop({'node',      't','d','f_a','disp_x',      'disp_y'}).rename({'node_right':'node','disp_x_right':'disp_x','disp_y_right':'disp_y'})]) \
                .group_by('node').agg(pl.col('disp_x').sum(), pl.col('disp_y').sum())

    # Add the forces to the positions & prepare to loop again
    df_pos = df_pos.join(df_sums, on='node').rename({'disp_x':'dx','disp_y':'dy'}) \
                   .with_columns((pl.col('dx')**2 + pl.col('dy')**2).sqrt().alias('d')) \
                   .with_columns(pl.when(pl.col('d') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('d')).alias('d')) \
                   .with_columns(pl.when(pl.col('dx').abs() > t).then(pl.lit(t)).otherwise(pl.col('dx')).alias('xmag'),
                                 pl.when(pl.col('dy').abs() > t).then(pl.lit(t)).otherwise(pl.col('dy')).alias('ymag')) \
                   .with_columns((pl.when(pl.col('s')).then(pl.col('x')).otherwise(pl.col('x') + pl.col('xmag')*pl.col('dx')/pl.col('d'))).alias('x'),
                                 (pl.when(pl.col('s')).then(pl.col('y')).otherwise(pl.col('y') + pl.col('ymag')*pl.col('dy')/pl.col('d'))).alias('y')) \
                   .drop({'dx','dy','d','xmag','ymag'})
    
    # Cool the temperature
    t *= 0.99

positions.append(df_pos)

pos = {}
for i in range(len(df_pos)): pos[df_pos['node'][i]] = (df_pos['x'][i], df_pos['y'][i])

# Make an animation (modifcation from the first cell)
_dur_ = '5s'
x0, y0, x1, y1 = positions[0]['x'].min(), positions[0]['y'].min(), positions[0]['x'].max(), positions[0]['y'].max()
for _pos_ in positions: x0, y0, x1, y1 = min(x0, _pos_['x'].min()), min(y0, _pos_['y'].min()), max(x1, _pos_['x'].max()), max(y1, _pos_['y'].max())
svg = [f'<svg x="0" y="0" width="256" height="256" viewBox="{x0} {y0} {x1-x0} {y1-y0}">']
svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="white" />')
node_positions = {}
for _node_ in g.nodes: node_positions[_node_] = []
for _pos_ in positions:
    for i in range(len(_pos_)):
        _node_, _x_, _y_ = _pos_['node'][i], _pos_['x'][i], _pos_['y'][i]
        node_positions[_node_].append((_x_, _y_))
for _node_ in g.nodes:
    for _nbor_ in g.neighbors(_node_):
        _x1_, _y1_ = node_positions[_node_][0]
        _x2_, _y2_ = node_positions[_nbor_][0]
        x1_strs, y1_strs = [], []
        for _x_, _y_ in node_positions[_node_]: x1_strs.append(f'{_x_}'), y1_strs.append(f'{_y_}')
        x2_strs, y2_strs = [], []
        for _x_, _y_ in node_positions[_nbor_]: x2_strs.append(f'{_x_}'), y2_strs.append(f'{_y_}')
        svg.append(f'<line x1="{_x1_}" y1="{_y1_}" x2="{_x2_}" y2="{_y2_}" stroke="black" stroke-width="0.2">')
        svg.append(f'<animate attributeName="x1" values="{";".join(x1_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="y1" values="{";".join(y1_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="x2" values="{";".join(x2_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="y2" values="{";".join(y2_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append('</line>')

for _node_ in node_positions:
    _x_, _y_ = node_positions[_node_][0]
    svg.append(f'<circle cx="{_x_}" cy="{_y_}" r="2.0" fill="blue">')
    x_strs, y_strs = [], []
    for _x_, _y_ in node_positions[_node_]: x_strs.append(f'{_x_}'), y_strs.append(f'{_y_}')
    svg.append(f'<animate attributeName="cx" values="{";".join(x_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="cy" values="{";".join(y_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</circle>')
svg.append('</svg>')

rt.tile([''.join(svg), rt.link(df, [('fm','to')], pos=pos)])

In [ ]:
#
# "Graph Drawing by Force-directed Placement"
# Fruchterman & Reingold
# Sofftware -- Practice and Experience, Vol. 21 (1 1), 1129-1164 (November 1991)
#
# Figure 1 (Force-directed Placement)
#
# Almost the original ... but removed the boundary constraints
#
pos_history = {}
for _node_ in g.nodes: pos_history[_node_] = []

t    = 3.0
W, H = 256, 256
pos  = {}
for _node_ in g.nodes(): pos[_node_] = random.random()*W, random.random()*H
area = W*H
C   = 0.2 # paper says that C is found experimentally...
k   = C*sqrt(area/len(g.nodes()))
def f_a(z): return  z**2 / k # pseudocode says "x" vs "z" ... paper comments say "z"
def f_r(z): return  k**2 / z # paper comments differ on the sign

_iterations_ = 2*len(g.nodes())
for i in range(_iterations_):
    for _node_ in g.nodes: pos_history[_node_].append(pos[_node_])
    disp = {}
    # calculate repulsive forces
    for v in g.nodes():
        disp[v] = 0.0, 0.0
        for u in g.nodes():
            if u == v or g.has_edge(u, v): continue
            _diff_     = pos[v][0] - pos[u][0], pos[v][1] - pos[u][1]
            _diff_len_ = max(sqrt(_diff_[0]**2 + _diff_[1]**2), 0.001)
            disp[v]    = disp[v][0] + (_diff_[0]/_diff_len_) * f_r(_diff_len_), disp[v][1] + (_diff_[1]/_diff_len_) * f_r(_diff_len_)
    # calculate the attractive forces
    for e in g.edges():
        e_v, e_u, w = e[0], e[1], 1 if 'weight' not in g[e[0]][e[1]] else g[e[0]][e[1]]['weight']
        _diff_     = pos[e_v][0] - pos[e_u][0], pos[e_v][1] - pos[e_u][1]
        _diff_len_ = max(sqrt(_diff_[0]**2 + _diff_[1]**2), 0.001)
        disp[e_v]  = disp[e_v][0] - (_diff_[0]/(_diff_len_*w)) * f_a(_diff_len_), disp[e_v][1] - (_diff_[1]/(_diff_len_*w)) * f_a(_diff_len_)
        disp[e_u]  = disp[e_u][0] + (_diff_[0]/(_diff_len_*w)) * f_a(_diff_len_), disp[e_u][1] + (_diff_[1]/(_diff_len_*w)) * f_a(_diff_len_)
    # limit the maximum displacement to the temperature t
    # ... and then prevent from being displaced outside frame
    for v in g.nodes():
        disp_len   = max(sqrt(disp[v][0]**2 + disp[v][1]**2), 0.001)
        xmag, ymag = t if abs(disp[v][0]) > t else disp[v][0], t if abs(disp[v][1]) > t else disp[v][1]
        pos[v] = pos[v][0] + (disp[v][0]/disp_len)*xmag, pos[v][1] + (disp[v][1]/disp_len)*ymag
    t *= 0.99 # t = cool(t)
for _node_ in g.nodes: pos_history[_node_].append(pos[_node_])

x0, y0, x1, y1 = pos[v][0], pos[v][1], pos[v][0], pos[v][1]
for k in pos_history.keys():
    x0, y0 = min(x0, min(pos_history[k][0])), min(y0, min(pos_history[k][1]))
    x1, y1 = max(x1, max(pos_history[k][0])), max(y1, max(pos_history[k][1]))

# Convert it to an animation
svg = [f'<svg x="0" y="0" width="512" height="512" viewBox="{x0} {y0} {x1-x0} {y1-y0}">']
svg.append(f'<rect x="{x0}" y="{y0}"  width="{x1-x0}" height="{y1-y0}" fill="white" />')
for _edge_ in g.edges:
    u, v = _edge_[0], _edge_[1]
    svg.append(f'<line x1="{pos[u][0]}" y1="{pos[u][1]}" x2="{pos[v][0]}" y2="{pos[v][1]}" stroke="black" stroke-width="0.2">')
    svg.append(f'<animate attributeName="x1" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[u]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="y1" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[u]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="x2" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[v]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="y2" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[v]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</line>')
for _node_ in g.nodes:
    svg.append(f'<circle cx="{pos_history[_node_][0][0]}" cy="{pos_history[_node_][0][1]}" r="1.2" fill="blue">')
    svg.append(f'<animate attributeName="cx" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[_node_]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="cy" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[_node_]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</circle>')
svg.append('</svg>')

_lu_ = {'fm':[], 'to':[]}
for edge in g.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
df = pl.DataFrame(_lu_)
rt.tile([''.join(svg), rt.link(df, [('fm','to')],pos)])